In [72]:
import pymysql
import numpy as np
import pandas as pd
import csv
import streamlit as st
import time

import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go

import datetime
import warnings
warnings.filterwarnings('ignore')

In [2]:
connect = pymysql.connect(host='localhost', user='root', password= '4643', db='danawa', charset='utf8mb4')
cur = connect.cursor()

def make_csv(table, condition, save_file_name):
    
    columns = []
    sql = "SHOW FULL COLUMNS FROM {}".format(table)
    cur.execute(sql)
    rows = cur.fetchall()
    
    for i in range(len(rows)):
        columns.append(rows[i][0])
    
    
    sql = 'SELECT * FROM {} WHERE {}'.format(table, condition)
    cur.execute(sql)
    
    rows = cur.fetchall()
    
    rows = list(rows)
    for row in range(len(rows)):
        rows[row] = list(rows[row])
    
    for i in range(len(rows)):
        rows[i][0] = rows[i][0].strftime("%Y-%m-%d")
    
    f = open(save_file_name, 'w', encoding = 'cp949', newline = '')
    wr = csv.writer(f)
    
    wr.writerow(columns)
    
    for i in range(len(rows)):
        wr.writerow(rows[i])
    
    f.close()
    connect.close()
    
    return f

## Modify above code

In [3]:
def import_data(table, start_date, end_date):
    connect = pymysql.connect(host = 'localhost', user = 'root', password = '4643', db = 'danawa', charset = 'utf8mb4')
    cur = connect.cursor()
    
    col_sql = "SHOW FULL COLUMNS FROM {}".format(table)
    cur.execute(col_sql)
    col_info = cur.fetchall()
    
    columns = [col[0] for col in col_info]
    
    sql = "SELECT * FROM {} WHERE DATE(CRAWL_DATE) >= '{}' AND DATE(CRAWL_DATE) <= '{}'".format(table,start_date, end_date)
    print(sql)
    
    cur.execute(sql)
    result = cur.fetchall()
    
    connect.close()
    
    df = pd.DataFrame(result, columns = columns)
    
    return df

In [67]:
df = import_data(table = 'cpu', start_date = '2020-06-08', end_date = '2020-06-09')
df = df.loc[df['HOUR'] >= 11]
df.loc[df['NAME'] == '인텔 코어i9-10세대 10900 (코멧레이크S) (정품)']

SELECT * FROM cpu WHERE DATE(CRAWL_DATE) >= '2020-06-08' AND DATE(CRAWL_DATE) <= '2020-06-09'


,CRAWL_DATE,HOUR,RANKING,NAME,PRICE,RELEASE_DATE
50,2020-06-08,11,1,인텔 코어i9-10세대 10900 (코멧레이크S) (정품),637000,2020-05-01
75,2020-06-08,11,26,인텔 코어i9-10세대 10900 (코멧레이크S) (정품),637000,2020-05-01
100,2020-06-08,12,1,인텔 코어i9-10세대 10900 (코멧레이크S) (정품),637000,2020-05-01
125,2020-06-08,12,26,인텔 코어i9-10세대 10900 (코멧레이크S) (정품),637000,2020-05-01
175,2020-06-08,13,26,인텔 코어i9-10세대 10900 (코멧레이크S) (정품),637000,2020-05-01
200,2020-06-08,14,1,인텔 코어i9-10세대 10900 (코멧레이크S) (정품),649000,2020-05-01
225,2020-06-08,14,26,인텔 코어i9-10세대 10900 (코멧레이크S) (정품),649000,2020-05-01
275,2020-06-08,15,26,인텔 코어i9-10세대 10900 (코멧레이크S) (정품),649000,2020-05-01
300,2020-06-08,16,1,인텔 코어i9-10세대 10900 (코멧레이크S) (정품),649000,2020-05-01
325,2020-06-08,16,26,인텔 코어i9-10세대 10900 (코멧레이크S) (정품),649000,2020-05-01


In [68]:
df.drop_duplicates(subset = ["CRAWL_DATE", "HOUR", "NAME"],inplace = True)

In [69]:
df.loc[df['NAME'] == '인텔 코어i9-10세대 10900 (코멧레이크S) (정품)']

,CRAWL_DATE,HOUR,RANKING,NAME,PRICE,RELEASE_DATE
50,2020-06-08,11,1,인텔 코어i9-10세대 10900 (코멧레이크S) (정품),637000,2020-05-01
100,2020-06-08,12,1,인텔 코어i9-10세대 10900 (코멧레이크S) (정품),637000,2020-05-01
175,2020-06-08,13,26,인텔 코어i9-10세대 10900 (코멧레이크S) (정품),637000,2020-05-01
200,2020-06-08,14,1,인텔 코어i9-10세대 10900 (코멧레이크S) (정품),649000,2020-05-01
275,2020-06-08,15,26,인텔 코어i9-10세대 10900 (코멧레이크S) (정품),649000,2020-05-01
300,2020-06-08,16,1,인텔 코어i9-10세대 10900 (코멧레이크S) (정품),649000,2020-05-01
375,2020-06-08,17,26,인텔 코어i9-10세대 10900 (코멧레이크S) (정품),649000,2020-05-01


## Line plot

### About Price

In [82]:
tmp = pd.pivot_table(df, index = ['CRAWL_DATE', 'HOUR'], columns = 'NAME', values = 'PRICE').reset_index()

tmp_time_list = []

for i in range(len(tmp)):
    time = pd.to_datetime(tmp['HOUR'], format = '%H').astype(str).str.split(" ")[i][1]
    
    tmp_time_list.append(time)
    
tmp['DATE'] = tmp['CRAWL_DATE'].astype(str) + " " +tmp_time_list
tmp.drop(columns = ['CRAWL_DATE', 'HOUR'], inplace = True)
tmp.set_index("DATE", inplace = True)
tmp.columns.name = None

In [74]:
tmp.index

Index(['2020-06-08 11:00:00', '2020-06-08 12:00:00', '2020-06-08 13:00:00',
       '2020-06-08 14:00:00', '2020-06-08 15:00:00', '2020-06-08 16:00:00',
       '2020-06-08 17:00:00'],
      dtype='object', name='DATE')

In [85]:
fig = go.Figure()

for product in tmp.columns:
    fig.add_trace(go.Scatter(x = tmp.index, y = tmp[product], name = "{}".format(product)))
    
fig.show()

In [75]:
tmp['인텔 코어i9-10세대 10900 (코멧레이크S) (정품)'].tolist()

[637000, 637000, 637000, 649000, 649000, 649000, 649000]

## About Ranking

In [57]:
tmp = pd.pivot_table(df, index = ['CRAWL_DATE', 'HOUR'], columns = 'NAME', values = 'RANKING').reset_index()

tmp_time_list = []

for i in range(len(tmp)):
    time = pd.to_datetime(tmp['HOUR'], format = '%H').astype(str).str.split(" ")[i][1]
    
    tmp_time_list.append(time)
    
tmp['DATE'] = tmp['CRAWL_DATE'].astype(str) + " " +tmp_time_list
tmp.drop(columns = ['CRAWL_DATE', 'HOUR'], inplace = True)
tmp.set_index("DATE", inplace = True)
tmp.columns.name = None

In [58]:
tmp['인텔 코어i9-10세대 10900 (코멧레이크S) (정품)']

DATE
2020-06-08 11:00:00    13.5
2020-06-08 12:00:00    13.5
2020-06-08 13:00:00    26.0
2020-06-08 14:00:00    13.5
2020-06-08 15:00:00    26.0
2020-06-08 16:00:00    13.5
2020-06-08 17:00:00    26.0
Name: 인텔 코어i9-10세대 10900 (코멧레이크S) (정품), dtype: float64

In [18]:
a.columns

Index(['AMD 라이젠 스레드리퍼 3990X (캐슬 픽) (정품)', 'AMD 라이젠3-1세대 2200G (레이븐 릿지) (정품)',
       'AMD 라이젠3-2세대 3200G (피카소) (정품)', 'AMD 라이젠3-3세대 3100 (마티스) (정품)',
       'AMD 라이젠3-3세대 3300X (마티스) (정품)', 'AMD 라이젠5-2세대 2600 (피나클 릿지) (정품)',
       'AMD 라이젠5-2세대 2600X (피나클 릿지) (정품)', 'AMD 라이젠5-2세대 3400G (피카소) (멀티팩)',
       'AMD 라이젠5-2세대 3400G (피카소) (정품)', 'AMD 라이젠5-3세대 3500 (마티스) (멀티팩)',
       'AMD 라이젠5-3세대 3500X (마티스) (멀티팩)', 'AMD 라이젠5-3세대 3600 (마티스) (정품)',
       'AMD 라이젠5-3세대 3600X (마티스) (정품)', 'AMD 라이젠7-3세대 3700X (마티스) (정품)',
       'AMD 라이젠7-3세대 3800X (마티스) (정품)', 'AMD 라이젠9-3세대 3900X (마티스) (정품)',
       'AMD 라이젠9-3세대 3950X (마티스) (정품)', 'AMD 애슬론 200GE (레이븐 릿지) (정품)',
       'AMD 애슬론 3000G (레이븐 릿지) (정품)', '인텔 셀러론 G4930 (커피레이크-R) (정품)',
       '인텔 코어X-시리즈 i9-10920X (캐스케이드레이크) (정품)',
       '인텔 코어X-시리즈 i9-10980XE Extreme Edition (캐스케이드레이크) (정품)',
       '인텔 코어i3-10세대 10100 (코멧레이크S) (정품)', '인텔 코어i3-9세대 9100 (커피레이크-R) (정품)',
       '인텔 코어i3-9세대 9100F (커피레이크-R) (정품)', '인텔 코어i5-10세대 10400 (코멧레이크S) (정품)'

In [16]:
def LinePlotPreprocess(data, process_type):
    if process_type == 'price':
        line_df = pd.pivot_table(data, index = ['CRAWL_DATE', 'HOUR'], columns = 'NAME', values = 'PRICE').reset_index()
    
    elif process_type == 'rank':
        line_df = pd.pivot_table(data, index = ['CRAWL_DATE', 'HOUR'], columns = 'NAME', values = 'RANKING').reset_index()
    
    tmp_time_list = []
    for i in range(len(line_df)):
        time = pd.to_datetime(line_df['HOUR'], format = '%H').astype(str).str.split(" ")[i][1]
        tmp_time_list.append(time)

    line_df['DATE'] = line_df['CRAWL_DATE'].astype(str) + " " + tmp_time_list
    line_df.drop(columns = ['CRAWL_DATE', 'HOUR'], inplace = True)
    line_df.set_index("DATE", inplace = True)
    line_df.columns.name = None

    if process_type == 'rank':
        for col in line_df.columns:
            line_df[col] = line_df[col].astype(int)
    
    return line_df